# Computing scores for recombination
1. Compute the errors at the front, for each live seed
2. Compute the error matrix between demonstrations
3. Compute errors with respect to the goal image.

In [ ]:
import os
import copy
import time
import json
import shutil
import unittest
import subprocess
from pathlib import Path
import numpy as np

from scipy.spatial.transform import Rotation as R

from gym_grasping.envs.robot_sim_env import RobotSimEnv
from flow_control.demo.demo_episode_recorder import record_sim
from flow_control.runner import evaluate_control
from flow_control.servoing.module import ServoingModule
from flow_control.servoing.playback_env_servo import PlaybackEnvServo
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact, Layout
import seaborn as sns
from tqdm import tqdm
import getpass

%matplotlib inline

experiment = "multi_part"

def get_data_dir():
    username = getpass.getuser()
    if username == "argusm":
        return "/tmp/flow_experiments3"
    elif username == "nayakab":
        return "../tmp"

data_dir = get_data_dir()

root_dir = os.path.join(data_dir, experiment)

## Load all demonstrations 

In [ ]:
def get_recordings(directory):
    return sorted([os.path.join(directory, rec) for rec in os.listdir(directory) if os.path.isdir(os.path.join(directory, rec))])

recordings = get_recordings(root_dir)

In [ ]:
# Load the demonstration episodes
playbacks = [PlaybackEnvServo(rec) for rec in recordings[:]]

# Plot the demonstrations
%matplotlib notebook
fig, ax = plt.subplots(1,figsize=(8, 6))
fig.suptitle("Demonstration Frames")
ax.set_axis_off()
image_h = ax.imshow(playbacks[0].cam.get_image()[0])

def update(demo_index, frame_index):
    image = playbacks[demo_index][frame_index].cam.get_image()[0]
    image_h.set_data(image)
    fig.canvas.draw_idle()
    print("wp_name:", playbacks[demo_index][frame_index].get_info()["wp_name"])
    fg_mask = playbacks[demo_index].get_fg_mask()
    if fg_mask is not None:
        print("percent fg:", np.mean(fg_mask)*100)
    
slider_w = widgets.IntSlider(min=0, max=len(playbacks)-1, step=1, value=0,
                             layout=Layout(width='70%'))
slider_i = widgets.IntSlider(min=0, max=200-1, step=1, value=0,
                             layout=Layout(width='70%'))

interact(update, demo_index=slider_w, frame_index=slider_i)

In [ ]:
def filter_demo(pb):
    return pb[-1].data['rew'] > 0 and np.mean(pb.get_fg_mask()) > 0.005

demo_good = [filter_demo(pb) for pb in playbacks]
good_demonstrations = np.where(demo_good)[0]
good_demonstrations = [int(x) for x in good_demonstrations]

In [ ]:
# Load demo segmentation file
demo_seg_file_2parts = f'{root_dir}/demo_parts_manual2.json'
demo_seg_file_3parts = f'{root_dir}/demo_parts_manual3.json'

fp = open(demo_seg_file_2parts)
demo_2parts = json.load(fp)

fp = open(demo_seg_file_3parts)
demo_3parts = json.load(fp)

live_indices = [int(key) for key in demo_2parts.keys()]

In [ ]:
# Servo Module
# Load Servoing Module
from flow_control.servoing.module import ServoingModule
control_config = dict(mode="pointcloud-abs-rotz", threshold=0.40)
servo_module = ServoingModule(recordings[0], control_config=control_config,
                              start_paused=False)

In [ ]:
def similarity_from_reprojection(live_rgb, demo_rgb, demo_mask, return_images=False):
    # evaluate the similarity via flow reprojection error
    flow = servo_module.flow_module.step(demo_rgb, live_rgb)
    warped = servo_module.flow_module.warp_image(live_rgb / 255.0, flow)
    diff = (warped - (demo_rgb / 255.0))
    error = np.linalg.norm((warped - (demo_rgb / 255.0)), axis=2) * demo_mask
    error = error.sum() / demo_mask.sum()
    mean_flow = np.linalg.norm(flow[demo_mask],axis=1).mean()
    if return_images:
        return error, mean_flow, flow, warped
    return error, mean_flow

In [ ]:
from sklearn.preprocessing import minmax_scale

def normalize_errors(errors, flows):
    errors_l = errors[demo_good]
    mean_flows_l = flows[demo_good]
    errors_norm = np.ones(errors.shape)
    w = 0.5
    errors_norm[demo_good] = np.mean((1*minmax_scale(errors_l), w*minmax_scale(mean_flows_l)),axis=0)/(1+w)
    return errors_norm

In [ ]:
demo_2parts

## Error matrix between demonstration parts 

In [ ]:
import seaborn as sns

error_matrix = np.ones((len(playbacks), len(playbacks)))
flow_matrix = np.zeros((len(playbacks), len(playbacks)))

def get_error_matrix(demo_parts, part_idx1, part_idx2, ):
    for k1, v1 in tqdm(demo_parts.items()):
        demo_i1 = int(k1)
        im1 = playbacks[demo_i1][v1[part_idx1]['end']].cam.get_image()[0]
        for k2, v2 in demo_parts.items():
            demo_i2 = int(k2)      
            im2 = playbacks[demo_i2][v2[part_idx2]['start']].cam.get_image()[0]
            mask2 = playbacks[demo_i2].fg_masks[v2[part_idx2]['start']]

            error, flow = similarity_from_reprojection(im1, im2, mask2)
            error_matrix[demo_i1, demo_i2] = error
            flow_matrix[demo_i1, demo_i2] = flow

    error_matrix_norm = normalize_errors(error_matrix, flow_matrix)
    
    return error_matrix_norm

error_matrix_2_part = get_error_matrix(demo_2parts, 0, 1)
np.savez(f"{root_dir}/error_matrix_2_part.npz", error_matrix_2_part)

error_matrix_3_part_1 = get_error_matrix(demo_3parts, 0, 1)
np.savez(f"{root_dir}/error_matrix_3_part_1.npz", error_matrix_3_part_1)

error_matrix_3_part_2 = get_error_matrix(demo_3parts, 1, 2)
np.savez(f"{root_dir}/error_matrix_3_part_2.npz", error_matrix_3_part_2)

sns.heatmap(1 - error_matrix_2_part)